In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List
import json

# 초기 데이터 생성

In [3]:
# 개발자 직무 30개
developer_roles = [
    (0, "Software Engineer"),
    (1, "Frontend Developer"),
    (2, "Backend Developer"),
    (3, "Full Stack Developer"),
    (4, "DevOps Engineer"),
    (5, "Cloud Engineer"),
    (6, "Data Engineer"),
    (7, "Machine Learning Engineer"),
    (8, "AI Engineer"),
    (9, "Embedded Software Engineer"),
    (10, "Mobile Developer"),
    (11, "iOS Developer"),
    (12, "Android Developer"),
    (13, "Game Developer"),
    (14, "Blockchain Developer"),
    (15, "Cybersecurity Engineer"),
    (16, "Systems Engineer"),
    (17, "Network Engineer"),
    (18, "Database Administrator"),
    (19, "QA Engineer"),
    (20, "Test Automation Engineer"),
    (21, "Site Reliability Engineer"),
    (22, "IoT Developer"),
    (23, "Big Data Engineer"),
    (24, "Business Intelligence Developer"),
    (25, "AR/VR Developer"),
    (26, "Firmware Engineer"),
    (27, "Solutions Architect"),
    (28, "Technical Support Engineer"),
    (29, "Software Architect")
]

In [ ]:
# 프롬프트
template = """
아래 예시 형식에 맞게 개발자를 희망하는 취업 준비생에게 필요한 프로젝트의 개요를 10개 작성해줘.

직무: {role}

<Example>
'title': '딥러닝 기반 영상-언어 융합 모델 개발', 'description': '영상과 텍스트 데이터를 융합하여 정보를 분석하고 예측하는 딥러닝 모델을 개발합니다. CNN과 RNN을 활용하여 다중 모달 데이터를 처리하고, 실험을 통해 모델의 성능을 검증합니다.', 'projectgoal': '영상-언어 융합 기술을 통해 다양한 응용 가능성을 탐색하고, 모델의 정확도를 높이는 것을 목표로 합니다.', 'techStacks': ['Python', 'TensorFlow', 'Keras', 'OpenCV'], 'qualifications': ['기계학습(딥러닝), 인공지능 전반에 대한 이론적, 기술적 이해', '영상-언어 융합 딥러닝 모델(CNNs, RNNs) 설계/개발 역량'], 'preferred_qualifications': ['기계학습(딥러닝), 이동체(차량, 비행체, 로봇), 인공지능 관련 최신 논문 이해 및 기법 재 구현 능력', '인공지능 관련 개발 지식/경험 보유 및 임베디드 경험'], 'userskillgaps': ['AI 신기술 평가 및 검증을 위한 실험 설계 및 실험 수행 역량', '확률 및 통계, 선형대수, 해석학 기반의 수학적 모델링에 대한 이해']
</Example>

<output_format>
{format_instructions}
</output_format>
"""

In [ ]:
# output 형식
os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project')
from models.project_info import Theme
class Themes(BaseModel):
    themes: List[Theme] = Field(..., description="생성된 프로젝트 개요의 리스트")

In [ ]:
# chain 생성
llm = ChatOpenAI(model="gpt-4o-mini")
prompt =  PromptTemplate(
    template=template,
    input_variables=['role']
)
parser = JsonOutputParser(pydantic_object=Themes)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

In [ ]:
# input data(프로젝트 개요) 생성
import asyncio
import json

# 직군 리스트, developer_roles 사용
# 비동기 호출을 위한 함수 정의
async def generate_project_data(role):
    input_data = {"role": role}
    result = await chain.ainvoke(input_data)
    print(f"Completed: {role}")
    return {role: result}

# 모든 직군에 대한 데이터를 비동기로 수집
async def main():
    tasks = [generate_project_data(role) for role in developer_roles]

    # 모든 작업을 병렬적으로 실행
    results = await asyncio.gather(*tasks)

    # 결과를 JSON 파일로 저장
    with open("/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_input.json", "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("All roles processed and saved!")

# 비동기 실행
await main()

In [ ]:
# output data(프로젝트 가이드라인) 생성
import asyncio
import json

os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project/services')
from chain_generator import gendetails_chain

# 직군 리스트, developer_roles를 사용

MAX_RETRIES = 3

# 비동기 호출을 위한 함수 정의
async def generate_output_data(theme, retries=0):
    chain = gendetails_chain()

    try:
        result = await chain.ainvoke(
            input={
                "recommended_project": theme
            }
        )
        print(f"Completed: {theme['title']}")
        return {theme['title']: result}
    
    except Exception as e:
        print(f"Error processing {theme['title']}: {e}")
        if retries < MAX_RETRIES:
            print(f"Retrying {theme['title']} ({retries + 1}/{MAX_RETRIES})")
            return await generate_output_data(theme, retries=retries + 1)
        else:
            print(f"Failed to process {theme['title']} after {MAX_RETRIES} attempts")
            return {theme['title']: None}

# 모든 직군에 대한 데이터를 비동기로 수집
async def main(i, role):
    # JSON 파일 경로
    input_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_input.json"
    output_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_output.json"

    # 파일 읽기
    with open(input_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    tasks = [generate_output_data(theme) for theme in data[i][role]["themes"]]
    
    # 모든 작업을 병렬적으로 실행
    results = await asyncio.gather(*tasks)

    # 기존 파일 불러오기 (덮어쓰지 않도록)
    try:
        with open(output_path, "r", encoding="utf-8") as json_file:
            existing_results = json.load(json_file)
    except FileNotFoundError:
        existing_results = []

    # 새로운 결과 추가
    existing_results.extend(results)

    # 결과를 JSON 파일로 저장
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(existing_results, json_file, ensure_ascii=False, indent=4)
    print(f"Completed processing for role: {role}")

# 비동기 실행
for i, role in developer_roles:
    await main(i, role)
    print(f"Completed processing for role: {role}")

In [7]:
# 생성된 데이터 확인 274개
import json
import pandas as pd

# JSON 파일 경로
file_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/"

# 파일 읽기
with open(file_path + "all_roles_input.json", "r", encoding="utf-8") as file:
    inputs = json.load(file)
with open(file_path + "all_roles_output.json", "r", encoding="utf-8") as file:
    outputs = json.load(file)

# 데이터프레임 생성
data = []
j = 0
for i, role in developer_roles:
    for theme in inputs[i][role]['themes']:
        data.append({
            'role': role,
            'input': theme,
            'output': outputs[j][theme['title']]
        })
        j += 1

df = pd.DataFrame(data)
df['difficultyLevel'] = df['output'].apply(lambda x: x.get('difficultyLevel', None))
print(df.shape)
df.head()

(274, 4)


,role,input,output,difficultyLevel
0,Software Engineer,"{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...","{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...",중급
1,Software Engineer,"{'title': '모바일 앱 개발', 'description': 'Flutter를...","{'title': '모바일 앱 개발', 'description': 'Flutter를...",중급
2,Software Engineer,"{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...","{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...",중급
3,Software Engineer,"{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...","{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...",중급
4,Software Engineer,"{'title': '게임 개발 프로젝트', 'description': 'Unity를...","{'title': '게임 개발 프로젝트', 'description': 'Unity를...",중급


# 난이도 변경(초급, 고급 추가)

In [ ]:
# 프롬프트
template = """
<Role>
넌 프로젝트 전문 컨설턴트야. 일반적인 프로젝트 주제를 구체화하고, 세부 단계를 구성하는데 특화되어 있어.
이전 프로젝트인 'prev_project'를 기준으로 사용자의 희망 난이도를 반영해서 프로젝트를 구체화 해줘.
</Role>

<instructions>
- Read through the all the below sections to get an understanding of the task.
- 프로젝트는 6단계로 나누고, 각 단계별 세부 Task를 포함하여 설명하세요.
- 세부 Task의 작업 단위는 최대한 작고 세세하게 작성하세요.
- 각 Task는 구체적인 예시를 포함해 명확히 작성하세요.
- 각 세부 Task마다 학습에 도움이 되는 검색 키워드를 같이 출력하세요. 예를 들어, Task의 내용이 '데이터 수집'이라면 'Kaggle', 'AIHub'와 같은 데이터 플랫폼을 같이 제공해줘.
- 사용자 혼자 두 달간 진행하는 프로젝트임을 감안해서 Task를 작성해
- 결과는 반드시 한글로 출력해야 합니다.
</instructions>

<prev_project>
{prev_project}
</prev_project>

<hope>
희망 난이도: {hopeLevel}
</hope>

<Output_Format>
{format_instructions}
</Output_Format>
"""

In [ ]:
# output 형식
os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project')
from models.project_info import Project

In [ ]:
# chain 생성
llm = ChatOpenAI(model="gpt-4o-mini")
prompt =  PromptTemplate(
    template=template,
    input_variables=['prev_project', 'hopeLevel']
)
parser = JsonOutputParser(pydantic_object=Project)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

In [ ]:
# 난이도 변경 데이터 생성
import asyncio
import json
from tqdm.asyncio import tqdm

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(10)

# 비동기 호출을 위한 함수 정의
async def generate_project_data(prev_project, hopeLevel):
    async with semaphore:  # 세마포어로 동시 호출 제한
        input_data = {"prev_project": prev_project, "hopeLevel": hopeLevel}
        result = await chain.ainvoke(input_data)
        print(f"Completed: {prev_project['title']} ({hopeLevel})")
        return result

# 쉽게, 어렵게 비동기 생성
async def main():
    tasks_easy = [generate_project_data(prev_project, '쉽게') for prev_project in df['output'].tolist()]
    tasks_hard = [generate_project_data(prev_project, '어렵게') for prev_project in df['output'].tolist()]
    tasks = tasks_easy + tasks_hard

    # 모든 작업을 병렬적으로 실행
    results = await tqdm.gather(*tasks, desc="Processing Projects", total=len(tasks))

    # 결과를 JSON 파일로 저장
    with open("/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/output_level.json", "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)
    print("Done!")

# 비동기 실행
await main()

In [ ]:
# 생성된 데이터 확인
import json
import pandas as pd

# JSON 파일 경로
file_path = "/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata"

# 파일 읽기
with open(file_path + "/output_level.json", "r", encoding="utf-8") as file:
    output_level = json.load(file)

# 데이터프레임 생성
output_level_df = pd.DataFrame({'output': output_level})
output_level_df['difficultyLevel'] = output_level_df['output'].apply(lambda x: x['difficultyLevel'])
output_level_df.head()

,output,difficultyLevel
0,"{'title': '기초 웹 애플리케이션 개발', 'description': 'HT...",쉬움
1,"{'title': '모바일 앱 개발 (쉬운 난이도)', 'description': ...",쉬움
2,"{'title': '쉬운 데이터 분석 및 시각화 프로젝트', 'description...",초급
3,"{'title': '웹 크롤러 및 데이터 수집 (쉬운 난이도)', 'descript...",쉬움
4,"{'title': '간단한 2D 게임 개발 프로젝트', 'description': ...",쉬움


In [ ]:
df_expanded = pd.concat([df] * 3, ignore_index=True)
df_expanded.loc[274:, 'output'] = output_level_df['output'].values
df_expanded.loc[274:, 'difficultyLevel'] = output_level_df['difficultyLevel'].values
df_base_level = df_expanded.copy()
df_base_level

,role,input,output,difficultyLevel
0,Software Engineer,"{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...","{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...",중급
1,Software Engineer,"{'title': '모바일 앱 개발', 'description': 'Flutter를...","{'title': '모바일 앱 개발', 'description': 'Flutter를...",중급
2,Software Engineer,"{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...","{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...",중급
3,Software Engineer,"{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...","{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...",중급
4,Software Engineer,"{'title': '게임 개발 프로젝트', 'description': 'Unity를...","{'title': '게임 개발 프로젝트', 'description': 'Unity를...",중급
...,...,...,...,...
817,Software Architect,"{'title': '블록체인 기반 분산 애플리케이션 개발', 'description...","{'title': '블록체인 기반 분산 애플리케이션 고급 개발', 'descript...",고급
818,Software Architect,"{'title': 'API 게이트웨이 설계 및 구현', 'description': ...","{'title': '고급 API 게이트웨이 설계 및 구현', 'description...",고급
819,Software Architect,"{'title': '데이터 파이프라인 구축', 'description': '데이터 ...","{'title': '고급 데이터 파이프라인 구축', 'description': '데...",고급
820,Software Architect,"{'title': '가상 현실(VR) 애플리케이션 개발', 'description'...","{'title': '고급 가상 현실(VR) 애플리케이션 개발', 'descripti...",고급


# 도메인 추가

In [ ]:
it_domains = [
    "금융(FinTech)",  # 모바일 결제, 암호화폐, 알고리즘 트레이딩
    "전자상거래(E-Commerce)",  # 아마존, 쿠팡, 쇼피파이 등 온라인 쇼핑 플랫폼
    "의료/헬스케어(HealthTech)",  # 원격 의료, AI 진단, 의료 데이터 분석
    "게임(Game Development)",  # 게임 엔진, 클라우드 게이밍, VR/AR 게임
    "자율주행(Autonomous Vehicles)",  # 테슬라, 자율주행 자동차, 드론 기술
    "사이버 보안(Cybersecurity)",  # 해킹 방지, 네트워크 보안, 데이터 암호화
    "클라우드 컴퓨팅(Cloud Computing)",  # AWS, Azure, GCP 같은 클라우드 인프라
    "사물인터넷(IoT, Internet of Things)",  # 스마트 홈, 스마트 시티, 산업 IoT
    "블록체인(Blockchain)",  # 암호화폐, 스마트 컨트랙트, 탈중앙화 애플리케이션
    "메타버스(Metaverse)",  # VR/AR 기반 가상 세계, 디지털 트윈, NFT 기반 가상 경제
]

In [ ]:
# 프롬프트
template = """
아래 프로젝트 개요에 IT 도메인을 적용해서 프로젝트 개요를 수정해줘.

프로젝트 개요: {input_project}
IT 도메인: {it_domain}

<output_format>
{format_instructions}
</output_format>
"""

In [ ]:
# output 형식
os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project')
from models.project_info import Theme

In [ ]:
# chain 생성
llm = ChatOpenAI(model="gpt-4o-mini")

prompt =  PromptTemplate(
    template=template,
    input_variables=['input_project', 'it_domain']
)
parser = JsonOutputParser(pydantic_object=Theme)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())
print(prompt.pretty_print())

chain = prompt | llm | parser


아래 프로젝트 개요에 IT 도메인을 적용해서 프로젝트 개요를 수정해줘.

프로젝트 개요: {input_project}
IT 도메인: {it_domain}

<output_format>
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"description": "프로젝트의 제목", "title": "Title", "type": "string"}, "description": {"description": "프로젝트에 대한 설명", "title": "Description", "type": "string"}, "projectgoal": {"description": "프로젝트의 목표", "title": "Projectgoal", "type": "string"}, "techStacks": {"description": "프로젝트에 사용되는 기술 스택 목록", "items": {"type": "string"}, "title": "Techstacks", "type": "array"}, "qualifications": {"description": "채용공고에서 언급된 필요 역량", "it

In [ ]:
# input_domain (도메인이 추가된 프로젝트 개요) 생성
import asyncio
import json
from tqdm.asyncio import tqdm  # tqdm의 비동기 지원 모듈

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(15)

# 도메인 리스트, it_domains 사용

# 비동기 호출을 위한 함수 정의
async def generate_project_data(input_project, it_domain):
    async with semaphore:
        input_data = {"input_project": input_project, "it_domain": it_domain}
        result = await chain.ainvoke(input_data)
        print(f"Completed: {input_project['title']} ({it_domain})")
        return result

# 모든 직군에 대한 데이터를 비동기로 수집
async def main():
    tasks = []
    for input_project in df['input'].tolist():
        for it_domain in it_domains:
            tasks.append(generate_project_data(input_project, it_domain))

    # 모든 작업을 병렬적으로 실행
    results = await tqdm.gather(*tasks, desc="Processing Projects", total=len(tasks))
    
    # 결과를 JSON 파일로 저장
    with open("/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/input_domain.json", "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)
    print("All roles processed and saved!")

# 비동기 실행
await main()

In [ ]:
# output data(도메인이 추가된 프로젝트 가이드라인) 생성
import asyncio
import json
from tqdm.asyncio import tqdm  # tqdm의 비동기 지원 모듈

os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project/services')
from chain_generator import gendetails_chain

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(10)

MAX_RETRIES = 3

# 비동기 호출을 위한 함수 정의
async def generate_output_data(input, index, retries=0):
    async with semaphore:
        chain = gendetails_chain()
        try:
            result = await chain.ainvoke(
                input={
                    "recommended_project": input
                }
            )
            # print(f"Completed: {input['title']}")
            return (index, result)  # ✅ 원래 순서(index)와 결과를 함께 반환
        
        except Exception as e:
            print(f"Error processing {input['title']}: {e}")
            if retries < MAX_RETRIES:
                print(f"Retrying {input['title']} ({retries + 1}/{MAX_RETRIES})")
                return await generate_output_data(input, retries=retries + 1)
            else:
                print(f"Failed to process {input['title']} after {MAX_RETRIES} attempts")
                return (index, {input['title']: None})  # ✅ 실패한 경우도 인덱스를 포함하여 반환

# 모든 직군에 대한 데이터를 비동기로 수집
async def main():
    # JSON 파일 경로
    input_path = "/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/input_domain.json"
    output_path = "/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/output_domain.json"

    # 파일 읽기
    with open(input_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    total_tasks = len(data)
    progress_bar = tqdm(total=total_tasks, desc="Processing Projects", unit="task", disable=True)

    results = [None] * total_tasks  # ✅ None을 채운 리스트를 미리 생성
    completed_count = 0

    # 비동기 작업 실행 및 진행도 업데이트
    tasks = [generate_output_data(d, i) for i, d in enumerate(data)]  # ✅ 원래 인덱스 포함

    for future in asyncio.as_completed(tasks):
        index, result = await future
        results[index] = result
        completed_count += 1
        progress_bar.update(1)

        # 10개 단위로 진행률 출력
        if completed_count % 10 == 0:
            progress_bar.n = completed_count  # ✅ tqdm 진행률을 수동으로 설정
            progress_bar.refresh()  # ✅ tqdm 수동 업데이트
            print(f"✅ {completed_count}/{total_tasks} tasks completed.")

    progress_bar.close()    

    # 결과를 JSON 파일로 저장
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

# 비동기 실행
await main()

In [ ]:
import pandas as pd
import json
data = json.load(open("/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/input_domain.json"))
output_data = json.load(open("/Users/no.2/Desktop/GitHub/cpplab-ai/test/gendata/output_domain.json"))
df_domain = pd.DataFrame(columns=['input'])
df_domain['input'] = data
df_domain['output'] = output_data
df_domain

,input,output
0,"{'title': '고급 핀테크 웹 애플리케이션 개발', 'description':...","{'title': '고급 핀테크 웹 애플리케이션 개발', 'description':..."
1,"{'title': '고급 전자상거래 웹 애플리케이션 개발', 'description...","{'title': '고급 전자상거래 웹 애플리케이션 개발', 'description..."
2,"{'title': '의료 웹 애플리케이션 개발', 'description': 'Re...","{'title': '의료 웹 애플리케이션 개발', 'description': 'Re..."
3,"{'title': '고급 게임 애플리케이션 개발', 'description': 'U...","{'title': '고급 게임 애플리케이션 개발', 'description': 'U..."
4,"{'title': '고급 자율주행 시스템 개발', 'description': 'Re...","{'title': '고급 자율주행 시스템 개발', 'description': 'Re..."
...,...,...
2735,"{'title': '사이버 보안 데이터 분석 플랫폼 개발', 'description...","{'title': '사이버 보안 데이터 분석 플랫폼 개발', 'description..."
2736,"{'title': '클라우드 기반 사물인터넷(IoT) 데이터 분석 플랫폼 개발', ...","{'title': '클라우드 기반 사물인터넷(IoT) 데이터 분석 플랫폼 개발', ..."
2737,"{'title': '사물인터넷(IoT) 데이터 분석 플랫폼 개발', 'descrip...","{'title': '사물인터넷(IoT) 데이터 분석 플랫폼 개발', 'descrip..."
2738,"{'title': '블록체인 기반 IoT 데이터 분석 플랫폼 개발', 'descri...","{'title': '블록체인 기반 IoT 데이터 분석 플랫폼 개발', 'descri..."
